In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('/home/prichter/Documents/selenobot/src/')

# Detection task

"Detection task" refers to the ability of the classifier to identify a sequence as either an erroneously-truncated selenoprotein, or a full-length non-selenoprotein. For this task, we used full-length negative normal proteins as negative test cases. The negative cases did *not* include full-length selenoproteins; we are basing this project on the observation that selenoproteins are misidentified, so full-length selenoproteins will not be present in the data we use to validate the model. The positive cases consisted of selenoproteins truncated at the *first* selenocysteine residue only. 

In [6]:
# Define the data and figure directories. 
DATA_DIR = '/home/prichter/Documents/selenobot/data/uniprot_2023_03/detect' 
FIGURE_DIR = '/home/prichter/Documents/selenobot/figures/'

In [8]:
from src.classifiers import EmbeddingClassifier, AacClassifier
from src.dataset import get_dataloader
from src.utils import csv_ids, csv_labels

import os
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# Paths where the train and test data are stored. 
train_path = os.path.join(DATA_DIR, 'train.csv')
test_path = os.path.join(DATA_DIR, 'test.csv')
val_path = os.path.join(DATA_DIR, 'val.csv')

# First load the data into a dataset object. 
train_dataloader = get_dataloader(train_path, batch_size=256)
val_dataloader = get_dataloader(val_path, batch_size=256)
# test_dataloader = get_dataloader(test_path, batch_size=1024)

## Testing the benchmark

In [10]:
# First, try training with no other weight, to get a sense of how many epochs to use.
model = AacClassifier()
# model = EmbeddingClassifier(1024)

I am curious about how the weight passed into the `WeightedBCELoss` will affect the performance on batches with selenoprotein relative to batches without selenoprotein. For now, I will test out different values on the `AacClassifier`, mostly to see if there *are* any loss function weights which might allow the model to decrease training loss on batches with and without selenoproteins simultaneously (or if this method of binary classification simply isn't sufficient). Based on some preliminary runs, it seems as though around 20 epochs is enough time for the `AacClassifier` to get about as good as it will get, so I will stick with this number for now. 

In [11]:
# aac_reporters = []
# for bce_loss_weight in [1, 10, 100, 1000, 10000, 100000]:
#     model.reset() # Reset the model weights. 
#     aac_reporter = model._train(train_dataloader, val=test_dataloader, epochs=1, lr=0.001, bce_loss_weight=bce_loss_weight)
#     aac_reporters.append(aac_reporter)
#     break

model.reset()
reporter = model._train(train_dataloader, val=val_dataloader, epochs=15, lr=0.001, bce_loss_weight=1) 

classifiers.Classifier.train_: 100%|██████████| 15/15 [00:15<00:00,  1.04s/it, val_acc=0.33]


In [15]:
import sklearn
import numpy as np
tn, fp, fn, tp = np.ravel(sklearn.metrics.confusion_matrix(*model.predict(test_dataloader)))

print('true negative', tn)
print('false positive', fp)
print('false negative', fn)
print('true positive', tp)

NameError: name 'test_dataloader' is not defined

In [ ]:
plot_train_losses(infos, path=FIGURE_DIR + 'emb_train_losses_pooled_batches_with_sec.png', title='EmbeddingClassifier pooled train losses on batches with selenoproteins', pool=False, sec_only=True)
plot_train_losses(infos, path=FIGURE_DIR + 'emb_train_losses_pooled.png', title='EmbeddingClassifier pooled train losses', pool=True, sec_only=False)
plot_train_losses(infos, path=FIGURE_DIR + 'emb_train_losses.png', title='EmbeddingClassifier train losses', pool=False, sec_only=False)


NameError: name 'plot_train_losses' is not defined